# WhisperTS joining JSON to SRT in all folders

In [21]:
import re,codecs,unidecode
import pandas as pd
import networkx as nx
import matplotlib.pylab as p

import json,glob,os
nMachine=os.uname()[1]
if "iMac" in nMachine:
    nMachine="iMac"
elif "MacBook" in nMachine:
    nMachine="MacBook"
print(nMachine)

iMac


In [22]:
def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')

In [23]:
pause=0.200

jsonReps=glob.glob("/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/%s/Json/*/*/"%nMachine)
jsonReps

['/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/BerloinSalome/enregistrement_10/',
 '/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/BerloinSalome/enregistrement_9/',
 '/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/ClavereauDarrieumerlouMarcheix/ECOLE PRIMAIRE LEOPOLD LACHAIZE 10/',
 '/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/ClavereauDarrieumerlouMarcheix/ECOLE PRIMAIRE LEOPOLD LACHAIZE 13/',
 '/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/ClavereauDarrieumerlouMarcheix/ECOLE PRIMAIRE LEOPOLD LACHAIZE 14/',
 '/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/ClavereauDarrieumerlouMarcheix/ECOLE PRIMAIRE LEOPOLD LACHAIZE 16/',
 '/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/ClavereauDarrieumerlouMarcheix/ECOLE PRIMAIRE LEOPOLD LACHAIZE 17/',
 '/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/ClavereauDarrieumerlouMarcheix/ECOLE PRIMAIRE LEOPOLD LACH

### Gestion des timestamps

In [24]:
def second2timestamp(seconds):
    sHours=int(seconds/3600)
    sMinutes=int((seconds-sHours*3600)/60)
    sSeconds=int((seconds-sHours*3600-sMinutes*60))
    sMilliseconds=int((seconds%1)*1000)
    return "%02d:%02d:%02d,%03d"%(sHours,sMinutes,sSeconds,sMilliseconds)

def timestamp2second(timestamp):
    m=re.match("(\d\d):(\d\d):(\d\d),(\d\d\d)",timestamp)
    if m:
        seconds=(int(m.group(1))*60+int(m.group(2)))*60+int(m.group(3))+float(m.group(4))/1000
        return seconds
    else:
        print("problème de timestamp")
        return

def shiftTime(timestamp,decal):
    return second2timestamp(timestamp2second(timestamp)+decal)

In [25]:
punct = list('''!()[]{};:-'"\,<>./?@#$%^&*_~''')
maxSize=8

def removePunct(texte):
    result=texte
    for p in punct:
        result=result.replace(p,"")
    return result

### Gestion des structures JSON de Whisper

In [26]:
def getWords(whisper):
    lWords=[]
    for segment in whisper["segments"]:
        for word in segment["words"]:
            lWords.append((second2timestamp(word["start"]),
                              second2timestamp(word["end"]),
                              word["text"],
                              word["confidence"]))        
    return [(t1,t2,w,c) for t1,t2,w,c in lWords if w!=""]

def getList(tWhisper):
    return [unidecode.unidecode(removePunct(w)).lower() for _,_,w,_ in tWhisper]

### Trouver la jointure

In [27]:
def findSample(sample,ref):
    lowSample=[t.lower() for t in sample]
    lowRef=[t.lower() for t in ref]
    i=0
    while i<len(ref)-len(sample):
        # print("ref",i)
        if lowSample==lowRef[i:i+len(sample)]:
            return i
        else:
            i+=1
    return None

### Collage des mots
1. on cherche la partie commune entre le nouveau segment et les sous-titres précédents
2. on peut régler la longueur de la partie commune pour optimiser le collage *maxSize*
3. on incrémente les temps de la nouvelle partie à partir du deuxième mot (pour éviter les erreurs de timestamp en début d'enregistrement)

In [35]:
def pasteWords(srt,whisper2):
    idx=None
    srtList=getList(srt)
    w2=getWords(whisper2)
    w2List=getList(w2)
    lenW2=len(w2)
    compSize=min(maxSize,lenW2)
    i=1
    print(compSize)
    while i<lenW2-compSize:
        sample=w2List[i:i+compSize]
        ref=srtList
        # print(sample)
        idx=findSample(sample,ref)
        if idx:
            print("found",i,idx)
            break
        else:
            i+=1
    if idx:
        print("found",i,idx,srtList[idx],srt[idx][0],w2List[i],w2[i][0])
        tDecal=timestamp2second(srt[idx+1][0])-timestamp2second(w2[i+1][0])
        print(second2timestamp(tDecal))
        newSrt=[]
        for n in range(i+1,lenW2):
            nWord=w2[n]
            newWord=(shiftTime(nWord[0],tDecal),shiftTime(nWord[1],tDecal),nWord[2],nWord[3])
            newSrt.append(newWord)
        print(srt[idx+1],newSrt[0])
        return srt[:idx]+newSrt
    else:
        print("problème")
        print()
        return None

In [40]:
def makeSRT(repJson):
    segments=len(glob.glob(repJson+"*.json"))
    json1=repJson+"extrait-00.json"
    with open(json1) as inFile:
        whisper1=json.load(inFile)
    srt=getWords(whisper1)
    srtList=getList(srt)
    for i in range(1,segments):
        # print(i)
        json2=repJson+"extrait-%02d.json"%i
        with open(json2) as inFile:
            whisper2=json.load(inFile)
        newSrt=pasteWords(srt,whisper2)
        if not newSrt:
            print("pb",i-1,i)
            return (repJson,True)
        else:
            srt=newSrt

    srtTours=[]
    srtMots=[]
    bTour=True
    prevEnd=0
    pStart=second2timestamp(0)
    pEnd=0
    
    for i,(start,end,mot,confidence) in enumerate(srt):
        # print(start,end,mot,confidence)
        tStart=timestamp2second(start)
        tEnd=timestamp2second(end)
        # print(start,tStart,prevEnd)
        if tStart-prevEnd>pause and prevEnd!=0:
            pEnd=second2timestamp(prevEnd)
            pTour=" ".join(srtMots)
            srtTours.append((pStart,pEnd,pTour))
            pStart=start
            prevEnd=tEnd
            srtMots=[mot]
            bPhrase=True
        else:
            srtMots.append(mot)
            prevEnd=tEnd
    pTour=" ".join(srtMots)
    srtTours.append((pStart,pEnd,pTour))

    srtLines=[]
    for i,(start,end,mot) in enumerate(srtTours):
        print(start,end,mot)
        n=i+1
        srtLines.append(str(n))
        srtLines.append(start+" --> "+end)
        srtLines.append(mot)
        srtLines.append("")
        
    with codecs.open(json1.replace(".json","-tours.srt"),"w",encoding="utf8") as outFile:
        for srtLine in srtLines:
            outFile.write(srtLine+"\n")
    print(repJson)
    print()
    return (repJson,False)

In [41]:
fails=[]
for repJson in jsonReps:
    print(repJson)
    if makeSRT(repJson):
        fails.append(repJson)

/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/BerloinSalome/enregistrement_10/
8
problème

pb 0 1
/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/BerloinSalome/enregistrement_9/
8
found 11 150
found 11 150 javais 00:01:17,939 javais 00:00:17,899
00:01:00,000
('00:01:18,480', '00:01:18,640', 'oublié', 0.993) ('00:01:18,480', '00:01:18,640', 'oublié', 0.992)
8
problème

pb 1 2
/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/ClavereauDarrieumerlouMarcheix/ECOLE PRIMAIRE LEOPOLD LACHAIZE 10/
00:00:00,000 00:00:04,740 Je m'appelle [*] Noah, j'ai 7 ans.
00:00:06,179 00:00:11,919 Et sur cette image, je vois un garçon [*] qui [*] a un ballon en [*] train de courir,
00:00:12,339 00:00:13,740 et son ballon est en coeur.
00:00:14,660 00:00:18,579 La deux petites [*] filles, elles écoutent la danse.
00:00:19,719 00:00:21,620 [*] La petite fille, elle court.
00:00:23,019 00:00:29,760 Un garçon qui a des [*] lunettes et une casquette, il s'y assit. I

In [42]:
fails

['/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/BerloinSalome/enregistrement_10/',
 '/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/BerloinSalome/enregistrement_9/',
 '/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/ClavereauDarrieumerlouMarcheix/ECOLE PRIMAIRE LEOPOLD LACHAIZE 10/',
 '/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/ClavereauDarrieumerlouMarcheix/ECOLE PRIMAIRE LEOPOLD LACHAIZE 13/',
 '/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/ClavereauDarrieumerlouMarcheix/ECOLE PRIMAIRE LEOPOLD LACHAIZE 14/',
 '/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/ClavereauDarrieumerlouMarcheix/ECOLE PRIMAIRE LEOPOLD LACHAIZE 16/',
 '/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/ClavereauDarrieumerlouMarcheix/ECOLE PRIMAIRE LEOPOLD LACHAIZE 17/',
 '/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/ClavereauDarrieumerlouMarcheix/ECOLE PRIMAIRE LEOPOLD LACH

In [ ]:
segments=len(glob.glob(repJson+"*.json"))
segments
pause=0.200
